SELF-ATTENTION

In [2]:
import numpy as np
import math

L, d_k, d_v = 4,8,8
q = np.random.randn(L, d_k)
k = np.random.randn(L, d_k)
v = np.random.randn(L, d_v)
q,k,v

(array([[ 2.75249785e-01,  4.83159148e-01,  4.66497395e-01,
         -2.37673038e+00,  6.18552254e-01,  1.53266532e+00,
         -1.04888246e+00, -1.62380375e-03],
        [-8.26393921e-02, -2.22246045e-02, -6.18850687e-01,
          1.59509308e+00, -2.98899854e-01,  4.70837132e-01,
         -8.93437088e-01,  3.08799844e-01],
        [ 7.85949434e-01,  4.61509339e-01,  1.37698676e+00,
         -3.73440967e-01,  4.68181686e-01, -8.24196672e-01,
          9.93433622e-01, -1.34819707e+00],
        [ 3.23789993e-01,  2.84369911e+00, -9.19288295e-01,
          1.41435015e+00, -8.06813482e-01, -2.25586791e-02,
          5.54345963e-01, -8.68484336e-01]]),
 array([[-0.55363748,  1.51504698,  0.48155705, -0.46345492, -1.5538429 ,
         -0.97751189, -0.66632258,  0.12906853],
        [ 1.21471272,  0.52868581, -0.74968218, -1.38319245, -0.71161034,
         -0.44771393,  0.88902941,  0.64562888],
        [-0.95525258,  0.62577711,  0.03178567,  1.06402966, -1.04426736,
          0.8527665 , 

In [11]:
np.matmul(q,k.T) ,np.matmul(q,k.T).var()

(array([[ 0.1451256 ,  1.46763999, -1.09142433, -0.09488614],
        [-0.38581654, -2.44753419,  3.10344523,  1.1352059 ],
        [ 0.3424722 ,  0.73153265, -2.83084106,  0.44414574],
        [ 3.8251437 ,  1.14593875,  3.29765745,  5.631308  ]]),
 4.677161490285577)

In [12]:
scaled = np.matmul(q,k.T)/math.sqrt(d_k)
scaled.var()

0.5846451862856971

MASKING

In [6]:
mask = np.tril(np.ones((L,L)))
mask

array([[1., 0., 0., 0.],
       [1., 1., 0., 0.],
       [1., 1., 1., 0.],
       [1., 1., 1., 1.]])

In [8]:
mask[mask == 0] = -np.infty
mask[mask == 1] = 0

In [9]:
mask 

array([[  0., -inf, -inf, -inf],
       [  0.,   0., -inf, -inf],
       [  0.,   0.,   0., -inf],
       [  0.,   0.,   0.,   0.]])

In [13]:
scaled + mask

array([[ 0.05130965,        -inf,        -inf,        -inf],
       [-0.13640675, -0.86533401,        -inf,        -inf],
       [ 0.12108221,  0.25863585, -1.00085345,        -inf],
       [ 1.35239253,  0.40515053,  1.16589797,  1.99096804]])

In [14]:
print(np.sum(np.exp(scaled + mask),axis = -1))
(np.exp(scaled + mask)) 

[ 1.05264879  1.29339862  2.79144538 15.89761569]


array([[1.05264879, 0.        , 0.        , 0.        ],
       [0.87248768, 0.42091094, 0.        , 0.        ],
       [1.1287177 , 1.29516208, 0.36756561, 0.        ],
       [3.86666557, 1.49952821, 3.208803  , 7.32261891]])

SOFTMAX

In [3]:
def softmax(x):
    return (np.exp(x).T / np.sum(np.exp(x),axis = -1)).T

In [15]:
attention = softmax(scaled + mask)

In [16]:
attention

array([[1.        , 0.        , 0.        , 0.        ],
       [0.67456982, 0.32543018, 0.        , 0.        ],
       [0.40434884, 0.46397543, 0.13167573, 0.        ],
       [0.24322299, 0.09432409, 0.20184178, 0.46061114]])

In [17]:
new_v = np.matmul(attention, v)
new_v

array([[ 0.12001635, -0.31246027,  0.16569375,  2.72159384,  1.04951631,
        -0.18897771,  1.62848842, -1.92552051],
       [ 0.02740988, -0.10331992, -0.16148782,  1.45575751,  0.66573607,
        -0.08960352,  1.46640624, -0.7001111 ],
       [-0.19942889, -0.07254669, -0.36196432,  0.8106    ,  0.36112735,
        -0.12568808,  1.26416013,  0.01454312],
       [-0.98167272, -1.20291367, -0.30090644,  0.61634265,  1.2057789 ,
        -0.60681985,  0.69550304,  0.4565355 ]])

In [4]:
def scaled_dot_product_attention(q,k,v,mask=None):
    d_k = q.shape[-1]
    scaled = np.matmul(q, k.T) / math.sqrt(d_k)
    if mask is not None:
        scaled = scaled + mask
    attention = softmax(scaled)
    out = np.matmul(attention, v)
    return out, attention

In [18]:
scaled_dot_product_attention(q,k,v,mask)

(array([[ 0.12001635, -0.31246027,  0.16569375,  2.72159384,  1.04951631,
         -0.18897771,  1.62848842, -1.92552051],
        [ 0.02740988, -0.10331992, -0.16148782,  1.45575751,  0.66573607,
         -0.08960352,  1.46640624, -0.7001111 ],
        [-0.19942889, -0.07254669, -0.36196432,  0.8106    ,  0.36112735,
         -0.12568808,  1.26416013,  0.01454312],
        [-0.98167272, -1.20291367, -0.30090644,  0.61634265,  1.2057789 ,
         -0.60681985,  0.69550304,  0.4565355 ]]),
 array([[1.        , 0.        , 0.        , 0.        ],
        [0.67456982, 0.32543018, 0.        , 0.        ],
        [0.40434884, 0.46397543, 0.13167573, 0.        ],
        [0.24322299, 0.09432409, 0.20184178, 0.46061114]]))

MULTIHEAD ATTENTION

In [19]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F

In [20]:
L = 4
batch_size = 1
input_dim = 512
d_model = 512
x = torch.randn((batch_size, L, input_dim))

In [21]:
x.size()

torch.Size([1, 4, 512])

In [22]:
qkv_layer = nn.Linear(input_dim, 3*d_model)
qkv = qkv_layer(x)
qkv.shape

torch.Size([1, 4, 1536])

In [23]:
num_heads = 8
head_dim = d_model // num_heads
qkv = qkv.reshape(batch_size, L, num_heads, 3*head_dim)

In [24]:
qkv.shape

torch.Size([1, 4, 8, 192])

In [25]:
qkv = qkv.permute(0,2,1,3)
qkv

tensor([[[[-0.0852,  0.1322,  0.2466,  ...,  1.0625, -0.2707,  0.3686],
          [-0.0132, -1.4978, -0.9129,  ...,  0.9324,  0.4472,  0.8599],
          [ 0.1751, -0.0577, -0.0960,  ...,  1.2166, -0.3852, -0.4915],
          [ 0.5527, -0.1625, -0.5184,  ...,  0.6852, -0.9270,  0.3845]],

         [[ 0.8466, -0.6219,  0.3334,  ...,  0.4208, -0.0808, -1.0024],
          [ 0.8024,  0.5604, -0.6831,  ..., -0.6170, -0.2889,  0.9852],
          [ 0.8055, -0.3160,  1.5276,  ..., -0.0138, -0.6528,  0.4051],
          [-0.2784,  0.3245, -0.6296,  ..., -0.6623, -0.3197,  0.8186]],

         [[-0.6355,  0.2266, -0.3077,  ...,  0.1214, -0.6224, -0.7657],
          [ 0.9627,  1.3713, -0.4762,  ..., -0.4577, -0.0143,  0.1391],
          [-1.0184,  0.1357,  0.5903,  ...,  0.1498, -0.4999, -0.0425],
          [ 0.1297,  0.2458,  0.2871,  ...,  0.2909,  0.5753, -0.4591]],

         ...,

         [[ 0.5632, -0.3538,  0.4080,  ...,  0.2881, -1.4450, -0.0399],
          [-0.4279,  0.7804, -0.2991,  ...,

In [26]:
qkv.shape

torch.Size([1, 8, 4, 192])

In [28]:
q, k, v = qkv.chunk(3, dim=-1)
q.shape

torch.Size([1, 8, 4, 64])

In [29]:
k.transpose(-2,-1).shape

torch.Size([1, 8, 64, 4])

In [30]:
d_k = q.size()[-1]
scaled = torch.matmul(q, k.transpose(-2,-1)) / math.sqrt(d_k)

In [31]:
scaled.shape

torch.Size([1, 8, 4, 4])

In [39]:
mask = torch.full(scaled.size(), float('-inf'))
mask = torch.triu(mask, diagonal=1)
mask

tensor([[[[0., -inf, -inf, -inf],
          [0., 0., -inf, -inf],
          [0., 0., 0., -inf],
          [0., 0., 0., 0.]],

         [[0., -inf, -inf, -inf],
          [0., 0., -inf, -inf],
          [0., 0., 0., -inf],
          [0., 0., 0., 0.]],

         [[0., -inf, -inf, -inf],
          [0., 0., -inf, -inf],
          [0., 0., 0., -inf],
          [0., 0., 0., 0.]],

         [[0., -inf, -inf, -inf],
          [0., 0., -inf, -inf],
          [0., 0., 0., -inf],
          [0., 0., 0., 0.]],

         [[0., -inf, -inf, -inf],
          [0., 0., -inf, -inf],
          [0., 0., 0., -inf],
          [0., 0., 0., 0.]],

         [[0., -inf, -inf, -inf],
          [0., 0., -inf, -inf],
          [0., 0., 0., -inf],
          [0., 0., 0., 0.]],

         [[0., -inf, -inf, -inf],
          [0., 0., -inf, -inf],
          [0., 0., 0., -inf],
          [0., 0., 0., 0.]],

         [[0., -inf, -inf, -inf],
          [0., 0., -inf, -inf],
          [0., 0., 0., -inf],
          [0., 0., 0.,

In [42]:
scaled+=mask
scaled

tensor([[[[-0.0735,    -inf,    -inf,    -inf],
          [ 0.0782,  0.2059,    -inf,    -inf],
          [-0.5030, -0.0269, -0.2396,    -inf],
          [-0.4872, -0.2374,  0.1777,  0.1478]],

         [[-0.3169,    -inf,    -inf,    -inf],
          [-0.2388,  0.0253,    -inf,    -inf],
          [-0.2687, -0.1476,  0.1627,    -inf],
          [-0.0392, -0.2156,  0.0171, -0.4682]],

         [[ 0.1863,    -inf,    -inf,    -inf],
          [-0.1263,  0.5383,    -inf,    -inf],
          [ 0.3032,  0.9043,  0.1395,    -inf],
          [ 0.2131,  0.4090, -0.2700,  0.1084]],

         [[ 0.1788,    -inf,    -inf,    -inf],
          [ 0.3041, -0.2295,    -inf,    -inf],
          [-0.0592,  0.0471, -0.2240,    -inf],
          [-0.0369,  0.2298,  0.1391, -0.0552]],

         [[-0.0401,    -inf,    -inf,    -inf],
          [ 0.1712,  0.1196,    -inf,    -inf],
          [-0.6292,  0.2859,  0.5819,    -inf],
          [ 0.3516, -0.2441,  0.5693, -0.1603]],

         [[-0.4056,    -inf,  

In [45]:
attention = F.softmax(scaled, dim=-1)
attention

tensor([[[[1.0000, 0.0000, 0.0000, 0.0000],
          [0.4681, 0.5319, 0.0000, 0.0000],
          [0.2557, 0.4116, 0.3327, 0.0000],
          [0.1635, 0.2099, 0.3179, 0.3086]],

         [[1.0000, 0.0000, 0.0000, 0.0000],
          [0.4343, 0.5657, 0.0000, 0.0000],
          [0.2726, 0.3077, 0.4197, 0.0000],
          [0.2819, 0.2363, 0.2982, 0.1836]],

         [[1.0000, 0.0000, 0.0000, 0.0000],
          [0.3397, 0.6603, 0.0000, 0.0000],
          [0.2722, 0.4966, 0.2312, 0.0000],
          [0.2678, 0.3258, 0.1652, 0.2412]],

         [[1.0000, 0.0000, 0.0000, 0.0000],
          [0.6303, 0.3697, 0.0000, 0.0000],
          [0.3378, 0.3757, 0.2865, 0.0000],
          [0.2232, 0.2914, 0.2662, 0.2192]],

         [[1.0000, 0.0000, 0.0000, 0.0000],
          [0.5129, 0.4871, 0.0000, 0.0000],
          [0.1459, 0.3643, 0.4898, 0.0000],
          [0.2947, 0.1624, 0.3663, 0.1766]],

         [[1.0000, 0.0000, 0.0000, 0.0000],
          [0.6260, 0.3740, 0.0000, 0.0000],
          [0.2571, 0.3

In [80]:
values = torch.matmul(attention, v)
values.shape

torch.Size([1, 8, 4, 64])

In [81]:
values

tensor([[[[-0.4777,  0.3902,  1.4209,  ...,  1.0625, -0.2707,  0.3686],
          [-0.4544, -0.2658, -0.0801,  ...,  0.9933,  0.1112,  0.6299],
          [-0.4466, -0.4839, -0.3586,  ...,  1.0602, -0.0133,  0.2846],
          [-0.2370, -0.3296, -0.4426,  ...,  0.9678, -0.3589,  0.2032]],

         [[-0.4836,  0.4204,  1.1629,  ...,  0.4208, -0.0808, -1.0024],
          [-0.4373, -0.0435,  0.3944,  ..., -0.1662, -0.1985,  0.1219],
          [-0.2621, -0.1597,  0.1489,  ..., -0.0809, -0.3849,  0.1999],
          [-0.2815, -0.1909,  0.1588,  ..., -0.1529, -0.3445,  0.2213]],

         [[ 0.3481,  0.8732, -0.4171,  ...,  0.1214, -0.6224, -0.7657],
          [ 0.3693,  0.4133, -0.0863,  ..., -0.2610, -0.2208, -0.1682],
          [ 0.2288,  0.4849,  0.0218,  ..., -0.1596, -0.2921, -0.1492],
          [ 0.0761,  0.0870,  0.0537,  ..., -0.0217, -0.1152, -0.2775]],

         ...,

         [[ 0.4565,  0.3440, -0.1666,  ...,  0.2881, -1.4450, -0.0399],
          [ 0.1780, -0.0540, -0.0328,  ...,

In [82]:
values = values.permute(0, 2, 1, 3)

In [83]:
values.shape

torch.Size([1, 4, 8, 64])

In [84]:
values = values.reshape(batch_size, L, num_heads*head_dim)

In [86]:
values

tensor([[[-0.4777,  0.3902,  1.4209,  ..., -1.0867, -0.4793,  0.7211],
         [-0.4544, -0.2658, -0.0801,  ..., -0.5343, -0.2098,  0.3576],
         [-0.4466, -0.4839, -0.3586,  ..., -0.4359, -0.1643, -0.0095],
         [-0.2370, -0.3296, -0.4426,  ..., -0.3046, -0.1781, -0.1269]]],
       grad_fn=<UnsafeViewBackward0>)

In [63]:
values.shape

torch.Size([1, 4, 512])

In [20]:
import torch.nn as nn
import torch
import math

class MultiHeadAttention(nn.Module):
    def __init__(self, input_dim, d_model, num_heads):
        super().__init__()
        self.input_dim = input_dim
        self.d_model = d_model
        self.num_heads = num_heads
        self.head_dim = d_model // num_heads
        self.qkv_layer = nn.Linear(input_dim, 3*d_model)
        self.softmax = nn.Softmax(dim = -1)
        self.linear = nn.Linear(d_model, d_model)

    def scaled_dot_product_attention(self,q,k,v,mask=None):
        d_k = q.shape[-1]
        k_t = k.transpose(-2,-1)
        scaled = torch.matmul(q, k_t) / math.sqrt(d_k)
        if mask is not None:
            scaled = scaled + mask
        attention = self.softmax(scaled)
        out = torch.matmul(attention, v)
        return out, attention
    
    def forward(self, x, mask = None):
        batch_size, L, input_dim = x.size()
        qkv = self.qkv_layer(x) #1536
        qkv = qkv.reshape(batch_size, L, self.num_heads, 3*self.head_dim)
        qkv = qkv.permute(0,2,1,3)
        q, k, v = qkv.chunk(3, dim=-1)
        out, attention = self.scaled_dot_product_attention(q,k,v,mask)
        out = out.permute(0,2,1,3)
        out = out.reshape(batch_size, L, self.d_model)
        out = self.linear(out)
        return out

In [22]:
input_dim = 1024
d_model = 512
num_heads = 8
batch_size = 30
L = 5
x = torch.randn(batch_size,L, input_dim)
model = MultiHeadAttention(input_dim, d_model, num_heads)
out = model.forward(x)
out.shape

torch.Size([30, 5, 512])

POSITIONAL EMBEDDING

In [23]:
import torch
import torch.nn as nn

L = 10
d_model = 6

In [24]:
even_i = torch.arange(0, d_model, 2).float()
even_i

tensor([0., 2., 4.])

In [25]:
even_denominator = torch.pow(10000, even_i/d_model)
even_denominator

tensor([  1.0000,  21.5443, 464.1590])

In [26]:
odd_i = torch.arange(1, d_model, 2).float()
odd_i

tensor([1., 3., 5.])

In [27]:
odd_denominator = torch.pow(10000, odd_i/d_model)
odd_denominator

tensor([   4.6416,  100.0000, 2154.4343])

In [33]:
position = torch.arange(L, dtype = torch.float).reshape(L,1)
position

tensor([[0.],
        [1.],
        [2.],
        [3.],
        [4.],
        [5.],
        [6.],
        [7.],
        [8.],
        [9.]])

In [35]:
even_PE = torch.sin(position/even_denominator)
odd_PE = torch.cos(position/odd_denominator)
even_PE, odd_PE

(tensor([[ 0.0000,  0.0000,  0.0000],
         [ 0.8415,  0.0464,  0.0022],
         [ 0.9093,  0.0927,  0.0043],
         [ 0.1411,  0.1388,  0.0065],
         [-0.7568,  0.1846,  0.0086],
         [-0.9589,  0.2300,  0.0108],
         [-0.2794,  0.2749,  0.0129],
         [ 0.6570,  0.3192,  0.0151],
         [ 0.9894,  0.3629,  0.0172],
         [ 0.4121,  0.4057,  0.0194]]),
 tensor([[ 1.0000,  1.0000,  1.0000],
         [ 0.9769,  0.9999,  1.0000],
         [ 0.9086,  0.9998,  1.0000],
         [ 0.7983,  0.9996,  1.0000],
         [ 0.6511,  0.9992,  1.0000],
         [ 0.4738,  0.9988,  1.0000],
         [ 0.2746,  0.9982,  1.0000],
         [ 0.0627,  0.9976,  1.0000],
         [-0.1522,  0.9968,  1.0000],
         [-0.3599,  0.9960,  1.0000]]))

In [36]:
stacked = torch.stack([even_PE, odd_PE], dim=2)
stacked.shape

torch.Size([10, 3, 2])

In [39]:
stacked = stacked.reshape(L, d_model)

In [40]:
stacked

tensor([[ 0.0000,  1.0000,  0.0000,  1.0000,  0.0000,  1.0000],
        [ 0.8415,  0.9769,  0.0464,  0.9999,  0.0022,  1.0000],
        [ 0.9093,  0.9086,  0.0927,  0.9998,  0.0043,  1.0000],
        [ 0.1411,  0.7983,  0.1388,  0.9996,  0.0065,  1.0000],
        [-0.7568,  0.6511,  0.1846,  0.9992,  0.0086,  1.0000],
        [-0.9589,  0.4738,  0.2300,  0.9988,  0.0108,  1.0000],
        [-0.2794,  0.2746,  0.2749,  0.9982,  0.0129,  1.0000],
        [ 0.6570,  0.0627,  0.3192,  0.9976,  0.0151,  1.0000],
        [ 0.9894, -0.1522,  0.3629,  0.9968,  0.0172,  1.0000],
        [ 0.4121, -0.3599,  0.4057,  0.9960,  0.0194,  1.0000]])

In [52]:
import torch
import torch.nn as nn
class PositionalEncoding(nn.Module):
    def __init__(self, d_model, L):
        super().__init__()
        self.d_model = d_model
        self.L = L
    def forward(self):
        even_i = torch.arange(0, d_model, 2)
        odd_i = torch.arange(1, d_model, 2)
        even_denominator = torch.pow(10000, even_i/self.d_model)
        odd_denominator = torch.pow(10000, odd_i/self.d_model)
        position = torch.arange(0,self.L).reshape(self.L, 1)
        even_PE = torch.sin(position/even_denominator)
        odd_PE = torch.cos(position/odd_denominator)
        stacked = torch.stack([even_PE, odd_PE], dim=2).reshape(self.L, self.d_model)
        return stacked

In [53]:
d_model = 512
L = 5
PEmodel = PositionalEncoding(d_model, L)
PE = PEmodel.forward()
print(PE)

tensor([[ 0.0000e+00,  1.0000e+00,  0.0000e+00,  ...,  1.0000e+00,
          0.0000e+00,  1.0000e+00],
        [ 8.4147e-01,  5.5522e-01,  8.2186e-01,  ...,  1.0000e+00,
          1.0366e-04,  1.0000e+00],
        [ 9.0930e-01, -3.8347e-01,  9.3641e-01,  ...,  1.0000e+00,
          2.0733e-04,  1.0000e+00],
        [ 1.4112e-01, -9.8103e-01,  2.4509e-01,  ...,  1.0000e+00,
          3.1099e-04,  1.0000e+00],
        [-7.5680e-01, -7.0591e-01, -6.5717e-01,  ...,  1.0000e+00,
          4.1465e-04,  1.0000e+00]])


In [54]:
PE.shape

torch.Size([5, 512])

LAYER_NORMALISATION

In [1]:
import torch
import torch.nn as nn

In [2]:
inputs = torch.Tensor([[[0.2,0.1,0.3],[0.5,0.1,0.1]]])
B,S,E = inputs.size()
inputs = inputs.reshape(S,B,E)
inputs.size()

torch.Size([2, 1, 3])

In [5]:
parameter_shape = inputs.size()[-2:]
gamma = nn.Parameter(torch.ones(parameter_shape))
beta = nn.Parameter(torch.zeros(parameter_shape))

In [6]:
gamma.size()

torch.Size([1, 3])

In [7]:
dims = [-(i+1) for i in range(len(parameter_shape))]

In [8]:
mean = inputs.mean(dim= dims, keepdim =True)
mean.size()

torch.Size([2, 1, 1])

In [9]:
mean

tensor([[[0.2000]],

        [[0.2333]]])

In [10]:
var = ((inputs-mean)**2).mean(dim= dims, keepdim=True)
epsilon = 1e-5
std = (var + epsilon).sqrt()
std

tensor([[[0.0817]],

        [[0.1886]]])

In [12]:
y = (inputs-mean)/std
y

tensor([[[ 0.0000, -1.2238,  1.2238]],

        [[ 1.4140, -0.7070, -0.7070]]])

In [14]:
out = gamma*y + beta
out

tensor([[[ 0.0000, -1.2238,  1.2238]],

        [[ 1.4140, -0.7070, -0.7070]]], grad_fn=<AddBackward0>)

In [17]:
import torch
import torch.nn as nn

class LayerNormalisation():
    def __init__(self,parameters_shape, eps=1e-5):
        self.parameters_shape = parameters_shape
        self.eps = eps
        self.gamma = nn.Parameter(torch.ones(parameters_shape))
        self.beta = nn.Parameter(torch.zeros(parameters_shape))
    
    def forward(self, input):
        dims = [-(i+1) for i in range(len(self.parameters_shape))]
        # dims = [i for i in range(len(self.parameters_shape))]
        mean = inputs.mean(dim = dims, keepdim = True)
        print("mean:", mean, "\n")
        var = ((inputs - mean)**2).mean(dim= dims, keepdim=True)
        std = (var + self.eps).sqrt()
        print("std: ",std,"\n")
        y = (inputs-mean)/std
        print(y)
        out = self.gamma * y + self.beta
        return out

In [9]:
B = 2
S = 5
E = 8
inputs = torch.randn(B,S,E)
inputs

tensor([[[ 0.8991,  2.0463,  0.1889, -0.7382,  0.9180, -1.3116,  0.6804,
          -1.0859],
         [-1.6766,  2.4427, -0.6603,  1.0465,  0.4836, -1.4417,  0.5144,
          -2.2974],
         [ 0.7005,  0.7009, -0.7390, -0.7901,  0.0563,  0.3950, -1.2784,
           0.3706],
         [ 1.2577, -0.2019,  1.7982, -0.2082,  1.8063,  1.0871, -0.3185,
           2.4050],
         [-1.1118, -0.6608,  0.6784,  0.2754,  1.0659,  1.1715, -0.6905,
          -0.1357]],

        [[-0.6208,  0.2440, -1.6580, -0.6958,  1.5642, -1.1322, -0.2868,
          -1.7205],
         [-0.5570,  0.4115, -0.6271,  0.1636, -0.6262,  0.8286,  0.1182,
           0.5821],
         [ 0.2554, -0.4994,  1.5506,  0.7403,  1.2673,  0.1709, -0.1303,
           0.3953],
         [-0.5525, -0.2413, -0.7346,  0.4996,  0.0394, -0.5766,  0.7247,
           0.1104],
         [-0.1518,  0.4539,  0.2824, -0.5254, -0.4645, -2.2145,  0.2749,
          -0.4563]]])

In [18]:
layer_norm = LayerNormalisation(inputs.size())
layer_norm.forward(inputs)

mean: tensor([[[0.0481]]]) 

std:  tensor([[[0.9884]]]) 

tensor([[[ 0.8609,  2.0216,  0.1425, -0.7956,  0.8801, -1.3757,  0.6397,
          -1.1473],
         [-1.7449,  2.4227, -0.7168,  1.0101,  0.4406, -1.5072,  0.4718,
          -2.3730],
         [ 0.6601,  0.6605, -0.7963, -0.8480,  0.0083,  0.3510, -1.3421,
           0.3263],
         [ 1.2238, -0.2529,  1.7706, -0.2593,  1.7789,  1.0512, -0.3709,
           2.3846],
         [-1.1735, -0.7172,  0.6377,  0.2300,  1.0297,  1.1366, -0.7472,
          -0.1860]],

        [[-0.6768,  0.1982, -1.7262, -0.7526,  1.5339, -1.1942, -0.3388,
          -1.7893],
         [-0.6122,  0.3677, -0.6831,  0.1168, -0.6822,  0.7897,  0.0710,
           0.5403],
         [ 0.2098, -0.5539,  1.5202,  0.7003,  1.2335,  0.1242, -0.1805,
           0.3513],
         [-0.6076, -0.2928, -0.7919,  0.4568, -0.0088, -0.6321,  0.6845,
           0.0630],
         [-0.2022,  0.4105,  0.2371, -0.5802, -0.5186, -2.2892,  0.2295,
          -0.5103]]])


tensor([[[ 0.8609,  2.0216,  0.1425, -0.7956,  0.8801, -1.3757,  0.6397,
          -1.1473],
         [-1.7449,  2.4227, -0.7168,  1.0101,  0.4406, -1.5072,  0.4718,
          -2.3730],
         [ 0.6601,  0.6605, -0.7963, -0.8480,  0.0083,  0.3510, -1.3421,
           0.3263],
         [ 1.2238, -0.2529,  1.7706, -0.2593,  1.7789,  1.0512, -0.3709,
           2.3846],
         [-1.1735, -0.7172,  0.6377,  0.2300,  1.0297,  1.1366, -0.7472,
          -0.1860]],

        [[-0.6768,  0.1982, -1.7262, -0.7526,  1.5339, -1.1942, -0.3388,
          -1.7893],
         [-0.6122,  0.3677, -0.6831,  0.1168, -0.6822,  0.7897,  0.0710,
           0.5403],
         [ 0.2098, -0.5539,  1.5202,  0.7003,  1.2335,  0.1242, -0.1805,
           0.3513],
         [-0.6076, -0.2928, -0.7919,  0.4568, -0.0088, -0.6321,  0.6845,
           0.0630],
         [-0.2022,  0.4105,  0.2371, -0.5802, -0.5186, -2.2892,  0.2295,
          -0.5103]]], grad_fn=<AddBackward0>)

ENCODER

In [31]:
import math
import torch
import torch.nn as nn
class MultiHeadAttention(nn.Module):
    def __init__(self, d_model, num_heads):
        super().__init__()
        self.d_model = d_model
        self.num_heads = num_heads
        self.head_dim = d_model // num_heads
        self.qkv_layer = nn.Linear(d_model, 3*d_model)
        self.softmax = nn.Softmax(dim = -1)
        self.linear = nn.Linear(d_model, d_model)

    def scaled_dot_product_attention(self,q,k,v,mask=None):
        d_k = q.shape[-1]
        k_t = k.transpose(-2,-1)
        scaled = torch.matmul(q, k_t) / math.sqrt(d_k)
        if mask is not None:
            scaled = scaled + mask
        attention = self.softmax(scaled)
        out = torch.matmul(attention, v)
        return out, attention
    
    def forward(self, x, mask = None):
        batch_size, L, d_model = x.size()
        qkv = self.qkv_layer(x) #1536
        qkv = qkv.reshape(batch_size, L, self.num_heads, 3*self.head_dim)
        qkv = qkv.permute(0,2,1,3)
        q, k, v = qkv.chunk(3, dim=-1)
        out, attention = self.scaled_dot_product_attention(q,k,v,mask)
        out = out.permute(0,2,1,3)
        out = out.reshape(batch_size, L, self.d_model)
        out = self.linear(out)
        return out

class LayerNormalisation(nn.Module):
    def __init__(self,parameters_shape,eps=1e-5):
        super().__init__()
        self.parameters_shape = parameters_shape
        self.eps = eps
        self.gamma = nn.Parameter(torch.ones(parameters_shape))
        self.beta = nn.Parameter(torch.zeros(parameters_shape))

    def forward(self, inputs):
        dims = [-(i+1) for i in range(len(self.parameters_shape))]
        mean = inputs.mean(dim=dims, keepdim=True)
        var = ((inputs - mean)**2).mean(dim=dims, keepdim=True)
        std = (var + self.eps).sqrt()
        y = (inputs - mean)/std
        out = self.gamma * y + self.beta
        return out

class PositionWiseFeedForward(nn.Module):
    def __init__(self, d_model, hidden,drop_prob):
        super(PositionWiseFeedForward, self).__init__()
        self.layer1 = nn.Linear(d_model, hidden)
        self.layer2 = nn.Linear(hidden, d_model)
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(p = drop_prob)
    
    def forward(self, x):
        x = self.layer1(x)
        x = self.relu(x)
        x = self.dropout(x)
        x = self.layer2(x)
        return x

class EncoderLayer(nn.Module):
    def __init__(self, d_model, ffn_hidden, num_heads, drop_prob):
        super(EncoderLayer, self).__init__()
        self.attention = MultiHeadAttention(d_model, num_heads)
        self.norm1 = LayerNormalisation(parameters_shape=[d_model])
        self.dropout1 = nn.Dropout(p = drop_prob)
        self.ffn = PositionWiseFeedForward(d_model, ffn_hidden, drop_prob)
        self.norm2 = LayerNormalisation([d_model])
        self.dropout2 = nn.Dropout(p=drop_prob)
    
    def forward(self,x):
        residual_x =x
        x = self.attention(x, mask=None)
        x = self.dropout1(x)
        x = self.norm1(x+residual_x)
        residual_x = x
        x = self.ffn(x)
        x = self.dropout2(x)
        x = self.norm2(x+residual_x)
        return x

class Encoder(nn.Module):
    def __init__(self, d_model, ffn_hidden, num_heads, drop_prob, num_layers):
        super().__init__()
        self.layers = nn.Sequential(*[EncoderLayer(d_model, ffn_hidden, num_heads, drop_prob) for _ in range(num_layers)])
    
    def forward(self,x):
        x = self.layers(x)
        return x

In [21]:
d_model = 512
num_heads = 8
drop_prob =0.1
batch_size = 30
max_seq_len = 200
ffn_hidden = 2048
num_layers = 5

In [34]:
encoder = Encoder(d_model, ffn_hidden,num_heads, drop_prob,num_layers)
x = torch.rand(batch_size,max_seq_len,d_model)
print(x,"\n")
encoder.forward(x)

tensor([[[8.5165e-02, 7.8972e-02, 9.3994e-01,  ..., 5.4412e-01,
          6.7304e-01, 7.0475e-01],
         [6.3159e-01, 4.2271e-01, 4.1495e-01,  ..., 8.9289e-01,
          3.7069e-01, 7.1196e-01],
         [6.3638e-01, 9.4616e-01, 9.3812e-01,  ..., 5.0189e-01,
          8.0082e-01, 8.6827e-01],
         ...,
         [7.9618e-01, 3.9775e-01, 9.7279e-01,  ..., 2.8340e-01,
          5.8786e-02, 9.3491e-01],
         [7.7949e-01, 2.6950e-01, 3.6036e-01,  ..., 3.7793e-01,
          2.3244e-01, 5.1813e-01],
         [6.2434e-01, 2.9904e-01, 1.2090e-02,  ..., 4.7482e-02,
          1.0576e-01, 9.7183e-01]],

        [[8.3025e-02, 2.1868e-01, 7.6788e-01,  ..., 8.6482e-01,
          2.1982e-01, 2.8630e-01],
         [3.1720e-01, 6.7618e-01, 7.2256e-02,  ..., 1.4309e-02,
          8.5812e-01, 7.1306e-01],
         [2.9088e-01, 3.8686e-01, 4.7623e-01,  ..., 6.5938e-01,
          6.1995e-01, 8.1219e-01],
         ...,
         [5.2448e-01, 1.8297e-01, 2.1650e-02,  ..., 1.6950e-01,
          3.716

tensor([[[-1.4750, -1.8187,  1.5631,  ..., -0.3323,  0.8195, -0.3356],
         [ 0.1586, -1.1768, -0.2073,  ...,  1.3578,  0.2599,  0.5032],
         [ 0.7782,  0.3307,  0.4487,  ...,  0.4960,  1.0771,  1.0387],
         ...,
         [ 0.6858, -0.6414, -0.1338,  ...,  0.1199, -0.8019,  0.4714],
         [ 1.1247, -0.9763, -0.4771,  ..., -0.4158, -0.1121, -1.1807],
         [ 0.4370, -0.8591, -0.7835,  ...,  0.4159, -0.9847,  0.7713]],

        [[-1.0092, -0.9389, -0.1810,  ...,  1.4506, -0.7547, -1.1111],
         [ 0.6898, -0.0060, -1.9131,  ..., -0.1013,  0.8371,  0.7687],
         [-1.0107, -0.9244,  0.3137,  ...,  0.2466,  0.4909,  0.3349],
         ...,
         [-0.0746, -1.7969, -0.9578,  ..., -0.6509,  0.8754, -0.4254],
         [-0.7202, -1.6371,  1.1020,  ..., -0.6296,  1.1235,  0.3191],
         [-0.8193, -0.8729, -0.1591,  ...,  1.5081, -0.4655,  1.0633]],

        [[ 0.8899,  0.2377, -0.4629,  ...,  0.4525,  1.6371,  0.9102],
         [-1.5423, -0.6512,  0.5578,  ...,  0

DECODER

In [57]:
import math
import torch
import torch.nn as nn

class MultiHeadCrossAttention(nn.Module):
    def __init__(self, d_model, num_heads):
        super().__init__()
        self.d_model = d_model
        self.num_heads = num_heads
        self.head_dim = d_model // num_heads
        self.kv_layer = nn.Linear(d_model, 2*d_model)
        self.qlayer = nn.Linear(d_model,d_model)
        self.softmax = nn.Softmax(dim = -1)
        self.linear = nn.Linear(d_model, d_model)

    def scaled_dot_product_attention(self,q,k,v,mask=None):
        d_k = q.shape[-1]
        k_t = k.transpose(-2,-1)
        scaled1 = torch.matmul(q, k_t) / math.sqrt(d_k)
        if mask is not None:
            scaled1 = scaled1 + mask
        attention = self.softmax(scaled1)
        out = torch.matmul(attention, v)
        return out, attention
    
    def forward(self, x,y, mask = None):
        batch_size, L, d_model = x.size()
        kv = self.kv_layer(x) #1024
        q = self.qlayer(y) #512
        kv = kv.reshape(batch_size, L, self.num_heads, 2*self.head_dim)
        q = q.reshape(batch_size, L, self.num_heads, self.head_dim)
        kv = kv.permute(0,2,1,3)
        q = q.permute(0,2,1,3)
        k, v = kv.chunk(2, dim=-1)
        out, attention = self.scaled_dot_product_attention(q,k,v,mask)
        out = out.permute(0,2,1,3)
        out = out.reshape(batch_size, L, self.d_model)
        out = self.linear(out)
        return out
    
class MultiHeadAttention(nn.Module):
    def __init__(self, d_model, num_heads):
        super().__init__()
        self.d_model = d_model
        self.num_heads = num_heads
        self.head_dim = d_model // num_heads
        self.qkv_layer = nn.Linear(d_model, 3*d_model)
        self.softmax = nn.Softmax(dim = -1)
        self.linear = nn.Linear(d_model, d_model)

    def scaled_dot_product_attention(self,q,k,v,mask=None):
        d_k = q.shape[-1]
        k_t = k.transpose(-2,-1)
        scaled = torch.matmul(q, k_t) / math.sqrt(d_k)
        if mask is not None:
            scaled = scaled + mask
        attention = self.softmax(scaled)
        out = torch.matmul(attention, v)
        return out, attention
    
    def forward(self, x, mask = None):
        batch_size, L, d_model = x.size()
        qkv = self.qkv_layer(x) #1536
        qkv = qkv.reshape(batch_size, L, self.num_heads, 3*self.head_dim)
        qkv = qkv.permute(0,2,1,3)
        q, k, v = qkv.chunk(3, dim=-1)
        out, attention = self.scaled_dot_product_attention(q,k,v,mask)
        out = out.permute(0,2,1,3)
        out = out.reshape(batch_size, L, self.d_model)
        out = self.linear(out)
        return out

class PositionWiseFeedForward(nn.Module):
    def __init__(self, d_model, hidden,drop_prob):
        super(PositionWiseFeedForward, self).__init__()
        self.layer1 = nn.Linear(d_model, hidden)
        self.layer2 = nn.Linear(hidden, d_model)
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(p = drop_prob)
    
    def forward(self, x):
        x = self.layer1(x)
        x = self.relu(x)
        x = self.dropout(x)
        x = self.layer2(x)
        return x


class LayerNormalisation(nn.Module):
    def __init__(self,parameters_shape,eps=1e-5):
        super().__init__()
        self.parameters_shape = parameters_shape
        self.eps = eps
        self.gamma = nn.Parameter(torch.ones(parameters_shape))
        self.beta = nn.Parameter(torch.zeros(parameters_shape))

    def forward(self, inputs):
        dims = [-(i+1) for i in range(len(self.parameters_shape))]
        mean = inputs.mean(dim=dims, keepdim=True)
        var = ((inputs - mean)**2).mean(dim=dims, keepdim=True)
        std = (var + self.eps).sqrt()
        y = (inputs - mean)/std
        out = self.gamma * y + self.beta
        return out

class DecoderLayer(nn.Module):
    def __init__(self, d_model,ffn_hidden, num_heads,  drop_prob):
        super(DecoderLayer,self).__init__()
        self.attention = MultiHeadAttention(d_model, num_heads)
        self.cross_attention = MultiHeadCrossAttention(d_model, num_heads)
        self.norm1 = LayerNormalisation(parameters_shape=[d_model])
        self.dropout1 = nn.Dropout(p = drop_prob)
        self.ffn = PositionWiseFeedForward(d_model, ffn_hidden, drop_prob)
        self.norm2 = LayerNormalisation([d_model])
        self.dropout2 = nn.Dropout(p=drop_prob)
        self.norm3 = LayerNormalisation([d_model])
        self.dropout3 = nn.Dropout(p=drop_prob)

    def forward(self,x,y,mask):
        residual_y = y
        y = self.attention(y,mask)
        y = self.dropout1(y)
        y = self.norm1(y + residual_y)
        residual_y = y
        y = self.cross_attention(x,y)
        y = self.dropout2(y)
        y = self.norm2(y + residual_y)
        residual_y = y
        y = self.ffn(y)
        y = self.dropout3(y)
        y = self.norm3(y + residual_y)
        return y

class SequentialDecoder(nn.Sequential):
    def forward(self, *inputs):
        x,y,mask = inputs
        for module in self._modules.values():
            y = module(x,y,mask)
        return y

class Decoder(nn.Module):
    def __init__(self, d_model, ffn_hidden, num_heads, drop_prob, num_layers):
        super().__init__()
        self.layers = SequentialDecoder(*[DecoderLayer(d_model, ffn_hidden, num_heads, drop_prob) for _ in range(num_layers)])
    
    def forward(self,x,y,mask):
        out = self.layers(x,y,mask)
        return out

In [37]:
d_model = 512
num_heads = 8
drop_prob =0.1
batch_size = 30
max_seq_len = 200
ffn_hidden = 2048
num_layers = 5

In [58]:
decoder = Decoder(d_model, ffn_hidden,num_heads, drop_prob,num_layers)
x = torch.randn(batch_size,max_seq_len,d_model)
y = torch.randn(batch_size,max_seq_len,d_model)
mask = torch.full([max_seq_len, max_seq_len], float('-inf'))
mask = torch.triu(mask, diagonal=1)
print(x,"\n")
print(y,"\n")
decoder.forward(x,y,mask)

tensor([[[-1.2075, -1.1127,  0.9051,  ..., -0.9427,  0.1867, -0.8870],
         [-1.6672, -0.0291, -1.1996,  ...,  0.4924, -0.8352, -0.5981],
         [ 0.1387, -1.2652, -0.7215,  ...,  0.8445, -0.4791, -0.0448],
         ...,
         [-0.1768,  0.3483,  1.1395,  ...,  1.0055,  0.0882,  0.8486],
         [ 1.5945, -0.4553, -1.2250,  ...,  0.4515, -0.6192, -0.1373],
         [ 0.2959, -0.6728, -2.0257,  ...,  1.3432, -2.4229, -0.8640]],

        [[ 0.8897,  0.4753,  0.7557,  ...,  1.5186, -1.5966,  0.1326],
         [ 0.4663, -1.8995, -0.3717,  ...,  1.0053, -0.3904, -0.4749],
         [-1.3406, -0.7369, -0.5987,  ...,  2.3235, -0.6220,  0.5518],
         ...,
         [ 0.9946,  0.7019,  0.6131,  ..., -1.1127,  1.2472,  1.0058],
         [ 0.2588,  0.4097, -0.4390,  ...,  1.7658,  0.5567,  0.5699],
         [ 0.0543,  0.0555, -0.2304,  ..., -1.0518, -0.9701, -1.2106]],

        [[ 1.4124,  0.1955,  0.6666,  ...,  0.0695,  0.5957,  0.8089],
         [ 1.8126,  0.3701,  0.5399,  ..., -2

tensor([[[-0.7983, -0.8367,  0.1433,  ...,  0.9049, -0.4009,  0.4072],
         [-0.2109,  0.0369,  1.0698,  ...,  0.3962, -0.5952, -0.3968],
         [-0.8200,  0.6144, -0.6390,  ...,  0.4931, -0.6483,  0.2895],
         ...,
         [ 0.5056,  0.6122, -0.4024,  ..., -0.5370, -0.5390,  0.2195],
         [-0.3717,  1.4342,  0.0632,  ..., -0.8621, -0.1562, -0.9854],
         [ 0.9549, -0.7390, -0.8214,  ..., -1.1474, -0.6037, -1.2576]],

        [[-0.9005,  0.1768,  0.3746,  ..., -0.3882,  1.0456, -1.1290],
         [ 0.2105,  0.7157,  1.1502,  ...,  1.5634, -1.5876, -1.5184],
         [ 0.0791, -0.9320,  2.4979,  ...,  0.2909, -0.3986, -1.3682],
         ...,
         [-0.0782, -0.4802, -0.8915,  ...,  0.0947,  0.4195,  0.8013],
         [ 0.8723,  1.5424,  0.5017,  ..., -0.8229, -0.8532, -0.7936],
         [ 1.0685, -0.9782,  0.7418,  ..., -0.2761, -0.1252, -0.4062]],

        [[-0.4163,  0.6572,  3.8089,  ...,  0.8401, -1.4169, -0.0507],
         [ 0.2555,  1.6492,  1.8763,  ...,  1

In [59]:
consonants = ['क', 'ख', 'ग', 'घ', 'ङ', 'च', 'छ', 'ज', 'झ', 'ञ', 'ट', 
              'ठ', 'ड', 'ढ', 'ण', 'त', 'थ', 'द', 'ध', 'न', 'प', 'फ', 
              'ब', 'भ', 'म', 'य', 'र', 'ल', 'व', 'श', 'ष', 'स', 'ह']

vowels = ['ा', 'ि', 'ी', 'ु', 'ू', 'ृ', 'े', 'ै', 'ो', 'ौ', 'ं', 'ः', '']

# Generate all combinations of consonants and vowels
word_set = set()
for consonant in consonants:
    for vowel in vowels:
        if vowel:
            word_set.add(consonant + vowel)
        else:
            word_set.add(consonant)

print("Set of words:", word_set)


Set of words: {'धः', 'चु', 'सौ', 'ङो', 'दे', 'सं', 'चृ', 'ढो', 'छी', 'नृ', 'खि', 'बे', 'खे', 'ज', 'ठु', 'ङि', 'यि', 'चो', 'ति', 'फा', 'फ', 'था', 'झे', 'ङे', 'षं', 'बु', 'लौ', 'बा', 'लं', 'वा', 'फे', 'शो', 'झ', 'यी', 'ढा', 'जी', 'गी', 'ञा', 'हू', 'थे', 'टृ', 'दी', 'स', 'कू', 'बो', 'बि', 'खू', 'मः', 'शा', 'सः', 'ह', 'फो', 'झृ', 'को', 'भू', 'षा', 'धी', 'हं', 'ची', 'मो', 'कृ', 'ढि', 'डा', 'या', 'नि', 'ती', 'ञौ', 'पू', 'छे', 'थौ', 'यु', 'णे', 'भा', 'घः', 'शु', 'लृ', 'दै', 'णः', 'लि', 'ढौ', 'णै', 'टै', 'शः', 'ढे', 'नं', 'खृ', 'लू', 'फृ', 'जि', 'य', 'गै', 'फु', 'कि', 'गा', 'प', 'जु', 'भु', 'डि', 'ञु', 'थ', 'ढ', 'षः', 'ड', 'त', 'फै', 'ठौ', 'डं', 'ञः', 'भी', 'जः', 'दु', 'धि', 'ठ', 'पं', 'यं', 'ली', 'वृ', 'गौ', 'रः', 'णि', 'टि', 'दः', 'ठृ', 'धु', 'चः', 'झौ', 'छं', 'थै', 'ञं', 'गि', 'मी', 'कै', 'सा', 'मा', 'पृ', 'ले', 'झी', 'यः', 'मे', 'ब', 'ङं', 'वं', 'यै', 'झै', 'ङृ', 'नू', 'कु', 'बृ', 'दि', 'तै', 'तु', 'थो', 'छै', 'धो', 'णी', 'ढं', 'गृ', 'मौ', 'घं', 'छो', 'कं', 'घो', 'ढै', 'थी', 'ठः', 'णो', 'ब

In [30]:
START = '<START>'
PADDING = '<PADDING>'
END = '\n'

characters_hindi = [START,'अ', 'आ', 'इ', 'ई', 'उ', 'ऊ', 'ऋ', 'ए', 'ऐ', 'ओ', 'औ', 
                    'क', 'ख', 'ग', 'घ', 'ङ', 'च', 'छ', 'ज', 'झ', 'ञ', 'ट', 
                    'ठ', 'ड', 'ढ', 'ण', 'त', 'थ', 'द', 'ध', 'न', 'प', 'फ', 
                    'ब', 'भ', 'म', 'य', 'र', 'ल', 'व', 'श', 'ष', 'स', 'ह','ॉ',
                    'ा', 'ि', 'ी', 'ु', 'ू', 'ृ', 'े', 'ै', 'ो', 'ौ', 'ं', 
                    'ः', '्', 'ॐ', '।', '॥', 'ँ', '़', 'ऽ', '०', '१', '२', 
                    '३', '४', '५', '६', '७', '८', '९', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ',', '.', ':', '!',
                    '?', '(', ')', '‘', '’', '“', '”', '-', '_',' ', PADDING, END]


characters_english = [START,'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 
              'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z',
              'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 
              'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z',
              '.', ',', ':', ';', '!', '?', "'", '"', '(', ')', '[', ']', '{', '}', 
              '-', '_', '/', '\\', '@', '#', '$', '%', '^', '&', '*', '+', '=', '<', '>', 
              '|', '~', '`', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9',' ', PADDING, END]


In [31]:
index_to_hindi = {k:v for k,v in enumerate(characters_hindi)}
hindi_to_index = {v:k for k,v in enumerate(characters_hindi)}
index_to_english = {k:v for k,v in enumerate(characters_english)}
english_to_index = {v:k for k,v in enumerate(characters_english)}

In [5]:
with open('D:\\ML\\transformer_scratch\\en-hi\\train.en') as enfile:
    english = [next(enfile) for _ in range(10000)]

In [2]:
english[:10]

['In reply, Pakistan got off to a solid start.\n',
 'The European Union has seven principal decision-making bodies, its institutions: the European Parliament, the European Council, the Council of the European Union, the European Commission, the Court of Justice of the European Union, the European Central Bank and the European Court of Auditors.\n',
 'The Congress leader represents Sivaganga Lok Sabha segment from Tamil Nadu.\n',
 'Prompt the user about connection attempts\n',
 'Further, the Minister announced that Deposit Insurance and Credit Guarantee Corporation (DICGC) has been permitted to increase Deposit Insurance coverage to Rs.\n',
 'Therefore, brothers, be more diligent to make your calling and election sure. For if you do these things, you will never stumble.\n',
 'The review committee meeting chaired by the District Judges will be attended by Collectors, SPs, Superintendents of jails and Secretary of District Legal Services Authority.\n',
 'Police is present on the spot.\n',

In [6]:
with open('D:\\ML\\transformer_scratch\\en-hi\\train.hi', encoding = 'utf-8') as hifile:
    hindi =  [next(hifile) for _ in range(10000)]

In [4]:
hindi[:10]

['जिसके जवाब में पाक ने अच्छी शुरुआत की थी.\n',
 'यूरोपीय संघ के महत्वपूर्ण संस्थानों में यूरोपियन कमीशन, यूरोपीय संसद, यूरोपीय संघ परिषद, यूरोपीय न्यायलय एवं यूरोपियन सेंट्रल बैंक इत्यादि शामिल हैं।\n',
 'कांग्रेस नेता तमिलनाडु से शिवगंगा लोकसभा क्षेत्र का प्रतिनिधित्व करते हैं.\n',
 'संबंधन प्रयास के बारे में उपयोक्ता को प्रांप्ट करें\n',
 'वित्त मंत्री ने घोषणा कि जमा बीमा और ऋण गारंटी निगम (डीआईसीजीसी) को जमा राशि बीमा का दायरा, जो इस समय 1 लाख रुपये है उसे बढ़ाकर प्रति जमाकर्ता 5 लाख रुपये करने की अनुमति प्रदान कर दी गई है।\n',
 'इस कारण हे भाइयों, अपने बुलाए जाने, और चुन लिये जाने को सिद्ध करने का भली भांति यत्न करते जाओ, क्योंकि यदि ऐसा करोगे, तो कभी भी ठोकर न खाओगे।\n',
 'समिति में जिला एवं सत्र न्यायाधीश, कलेक्टर, पुलिस अधीक्षक व जिला विधिक सेवा प्राधिकरण के सचिव शामिल रहते हैं।\n',
 'पुलिस मौके पर मौजूद है।\n',
 'जम्मू एवं कश्मीर विधानसभा के लिए दूसरे चरण के तहत होने वाले मतदान के लिए प्रधानमंत्री नरेंद्र मोदी भारतीय जनता पार्टी (भाजपा) के पक्ष में शुक्रवार को यहां रैलियां कर

In [6]:
max(len(x) for x in english)

1099

In [7]:
max(len(x) for x in hindi)

1172

In [19]:
import numpy as np
np.percentile([len(x) for x in english], 95)

232.0

In [18]:
np.percentile([len(x) for x in hindi], 95)

226.0

In [6]:
max_sq_len = 200

In [32]:
def is_valid_token(sentence, vocab):
    for token in list(set(sentence)):
        if token not in vocab:
                # print(token)
                return False
    return True

def isvalidlength(sentence, max_sq_len):
    return len(list(sentence))< (max_sq_len-1)

valid_index = []
for i in range(10000):
    hsen, esen = hindi[i], english[i]
    if(is_valid_token(hsen,characters_hindi) and is_valid_token(esen, characters_english) and isvalidlength(esen,max_sq_len) and isvalidlength(hsen,max_sq_len)):
        valid_index.append(i)
print(len(valid_index))

7303


In [55]:
from torch.utils.data import Dataset
class TextDataset(Dataset):
    def __init__(self, english, hindi):
        self.english = english
        self.hindi = hindi
    def __len__(self):
        return len(self.english)
    def __getitem__(self,idx):
        return self.english[idx], self.hindi[idx]

In [56]:
dataset = TextDataset(english,hindi)

In [57]:
len(dataset)

10000

In [49]:
import torch
def tokenize(sentence, language_to_index, start_token=True):
    sentence_word_indicies = [language_to_index[token] for token in list(sentence)]
    if start_token:
        sentence_word_indicies.insert(0, language_to_index[START])
    # print(len(sentence_word_indicies))
    for _ in range(len(sentence_word_indicies), max_sq_len):
        sentence_word_indicies.append(language_to_index[PADDING])
    return torch.tensor(sentence_word_indicies)

In [37]:
sentence_word_indicies = [english_to_index[token] for token in list(english[0])]

In [41]:
range(len(sentence_word_indicies),100)

range(45, 100)

In [50]:
eng_tokenized, hn_tokenized = [], []
for i in valid_index:
    eng_sentence, hn_sentence = english[i], hindi[i]
    eng_tokenized.append( tokenize(eng_sentence, english_to_index, start_token=False) )
    # print( tokenize(eng_sentence, english_to_index, start_token=False) )
    hn_tokenized.append( tokenize(hn_sentence, hindi_to_index, start_token=True) )
    # print( tokenize(hn_sentence, hindi_to_index, start_token=True) )
eng_tokenized = torch.stack(eng_tokenized)
hn_tokenized = torch.stack(hn_tokenized)

In [65]:
eng_tokenized[0]

tensor([ 9, 40, 95, 44, 31, 42, 38, 51, 54, 95, 16, 27, 37, 35, 45, 46, 27, 40,
        95, 33, 41, 46, 95, 41, 32, 32, 95, 46, 41, 95, 27, 95, 45, 41, 38, 35,
        30, 95, 45, 46, 27, 44, 46, 53, 97, 96, 96, 96, 96, 96, 96, 96, 96, 96,
        96, 96, 96, 96, 96, 96, 96, 96, 96, 96, 96, 96, 96, 96, 96, 96, 96, 96,
        96, 96, 96, 96, 96, 96, 96, 96, 96, 96, 96, 96, 96, 96, 96, 96, 96, 96,
        96, 96, 96, 96, 96, 96, 96, 96, 96, 96, 96, 96, 96, 96, 96, 96, 96, 96,
        96, 96, 96, 96, 96, 96, 96, 96, 96, 96, 96, 96, 96, 96, 96, 96, 96, 96,
        96, 96, 96, 96, 96, 96, 96, 96, 96, 96, 96, 96, 96, 96, 96, 96, 96, 96,
        96, 96, 96, 96, 96, 96, 96, 96, 96, 96, 96, 96, 96, 96, 96, 96, 96, 96,
        96, 96, 96, 96, 96, 96, 96, 96, 96, 96, 96, 96, 96, 96, 96, 96, 96, 96,
        96, 96, 96, 96, 96, 96, 96, 96, 96, 96, 96, 96, 96, 96, 96, 96, 96, 96,
        96, 96])

In [83]:
NEG_INFTY = -1e9

def create_masks(eng_batch, hin_batch):
    num_sen = len(eng_batch)
    look_ahead_mask = torch.full([max_sq_len, max_sq_len],True)
    look_ahead_mask = torch.triu(look_ahead_mask, diagonal=1)
    encoder_padding_mask = torch.full([num_sen, max_sq_len, max_sq_len], False)
    decoder_padding_mask_self_attention = torch.full([num_sen, max_sq_len, max_sq_len], False)
    decoder_padding_mask_cross_attention = torch.full([num_sen, max_sq_len, max_sq_len], False)

    for i in range(num_sen):
        eng_sen_len, hin_sen_len = len(eng_batch[i]), len(hin_batch[i])
        eng_padding_char = np.arange(eng_sen_len, max_sq_len)
        hin_padding_char = np.arange(hin_sen_len, max_sq_len)
        encoder_padding_mask[i,:,eng_padding_char] = True
        encoder_padding_mask[i,eng_padding_char, :] = True
        decoder_padding_mask_self_attention[i, :, hin_padding_char] = True
        decoder_padding_mask_self_attention[i, hin_padding_char, :] = True
        decoder_padding_mask_cross_attention[i, :, eng_padding_char] = True
        decoder_padding_mask_cross_attention[i, hin_padding_char, :] = True

    
    encoder_self_attention_mask = torch.where(encoder_padding_mask, NEG_INFTY,0)
    decoder_self_attention_mask = torch.where(look_ahead_mask + decoder_padding_mask_self_attention, NEG_INFTY, 0)
    decoder_cross_attention_mask = torch.where(decoder_padding_mask_cross_attention, NEG_INFTY, 0)
    return encoder_self_attention_mask, decoder_self_attention_mask, decoder_cross_attention_mask

In [87]:
e,d1,d2 = create_masks(eng_tokenized,hn_tokenized)

In [97]:
d2

tensor([[[0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         ...,
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.]],

        [[0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         ...,
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.]],

        [[0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         ...,
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.]],

        ...,

        [[0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         ...,
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0., 

In [95]:
torch.matmul(torch.ones(1,200),(torch.matmul(e[0],torch.ones((200,1)))))

tensor([[0.]])

In [80]:
import numpy as np
np.arange(100, max_sq_len)

array([100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112,
       113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125,
       126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138,
       139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151,
       152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164,
       165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177,
       178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190,
       191, 192, 193, 194, 195, 196, 197, 198, 199])

In [78]:
torch.triu(torch.full([max_sq_len, max_sq_len],True),diagonal=1)

tensor([[False,  True,  True,  ...,  True,  True,  True],
        [False, False,  True,  ...,  True,  True,  True],
        [False, False, False,  ...,  True,  True,  True],
        ...,
        [False, False, False,  ..., False,  True,  True],
        [False, False, False,  ..., False, False,  True],
        [False, False, False,  ..., False, False, False]])

In [99]:
import torch
import torch.nn as nn
class SentenceEmbedding(nn.Module):
    def __init__(self, max_sq_len, d_model, language_to_index, START, END, PADDING):
        super().__init__()
        self.vocab_size = len(language_to_index)
        self.max_sq_len = max_sq_len
        self.embedding = nn.Embedding(self.vocab_size, d_model)
        self.language_to_index = language_to_index
        self.position_encoder = PositionalEncoding(d_model, max_sq_len)
        self.dropout = nn.Droupout(p=0.1)
        self.START = START
        self.END = END
        self.PADDING = PADDING
    
    def batch_tokenisation(self, batch, start_token = True, end_token= True):
        def tokenize(sentence, language_to_index, start_token=True):
            sentence_word_indicies = [language_to_index[token] for token in list(sentence)]
            if start_token:
                sentence_word_indicies.insert(0, language_to_index[START])
            # print(len(sentence_word_indicies))
            for _ in range(len(sentence_word_indicies), max_sq_len):
                sentence_word_indicies.append(language_to_index[PADDING])
            return torch.tensor(sentence_word_indicies)
        eng_tokenized, hn_tokenized = [], []
        for i in valid_index:
            eng_sentence, hn_sentence = english[i], hindi[i]
            eng_tokenized.append( tokenize(eng_sentence, english_to_index, start_token=False) )
            # print( tokenize(eng_sentence, english_to_index, start_token=False) )
            hn_tokenized.append( tokenize(hn_sentence, hindi_to_index, start_token=True) )
            # print( tokenize(hn_sentence, hindi_to_index, start_token=True) )
        eng_tokenized = torch.stack(eng_tokenized)
        hn_tokenized = torch.stack(hn_tokenized)

In [1]:
import math
import torch
import torch.nn as nn

def get_device():
    return torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

class PositionalEncoding(nn.Module):
    def __init__(self, d_model, L):
        super().__init__()
        self.d_model = d_model
        self.L = L
    def forward(self):
        even_i = torch.arange(0, d_model, 2).float()
        # odd_i = torch.arange(1, d_model, 2).float()
        even_denominator = torch.pow(10000, even_i/self.d_model)
        # odd_denominator = torch.pow(10000, odd_i/self.d_model)
        position = torch.arange(self.L).reshape(self.L, 1)
        even_PE = torch.sin(position/even_denominator)
        odd_PE = torch.cos(position/even_denominator)
        stacked = torch.stack([even_PE, odd_PE], dim=2).reshape(self.L, self.d_model)
        # print("Positional Encoding")
        return stacked

class SentenceEmbedding(nn.Module):
    def __init__(self, max_sq_len, d_model, language_to_index, START, END, PADDING):
        super().__init__()
        self.vocab_size = len(language_to_index)
        self.max_sq_len = max_sq_len
        self.embedding = nn.Embedding(self.vocab_size, d_model)
        self.language_to_index = language_to_index
        self.position_encoder = PositionalEncoding(d_model, max_sq_len)
        self.dropout = nn.Dropout(p=0.1)
        self.START = START
        self.END = END
        self.PADDING = PADDING
    
    def batch_tokenisation(self, batch, start_token = True, end_token= True):
        def tokenize(sentence, start_token=True, end_token= True):
            sentence_word_indicies = [self.language_to_index[token] for token in list(sentence)]
            if start_token:
                sentence_word_indicies.insert(0, self.language_to_index[START])
            if end_token:
                sentence_word_indicies.append(self.language_to_index[self.END])
            for _ in range(len(sentence_word_indicies), self.max_sq_len):
                sentence_word_indicies.append(self.language_to_index[PADDING])
            return torch.tensor(sentence_word_indicies)
        tokenized = []
        for i in batch:
            tokenized.append( tokenize(i, start_token=start_token, end_token = end_token) )
        tokenized = torch.stack(tokenized)
        return tokenized

    def forward(self, x, start_token, end_token): # sentence
        x  = self.batch_tokenisation(x, start_token, end_token)
        x = self.embedding(x)
        pos = self.position_encoder().to(get_device())
        x = self.dropout(x + pos)
        # print("SentenceEmbedding")
        return x

class MultiHeadCrossAttention(nn.Module):
    def __init__(self, d_model, num_heads):
        super().__init__()
        self.d_model = d_model
        self.num_heads = num_heads
        self.head_dim = d_model // num_heads
        self.kv_layer = nn.Linear(d_model, 2*d_model)
        self.qlayer = nn.Linear(d_model,d_model)
        self.softmax = nn.Softmax(dim = -1)
        self.linear = nn.Linear(d_model, d_model)

    def scaled_dot_product_attention(self,q,k,v,mask=None):
        d_k = q.shape[-1]
        k_t = k.transpose(-2,-1)
        scaled1 = torch.matmul(q, k_t) / math.sqrt(d_k)
        if mask is not None:
            scaled1 = (scaled1.permute(1,0,2,3) + mask).permute(1,0,2,3)
        attention = self.softmax(scaled1)
        out = torch.matmul(attention, v)
        return out, attention
    
    def forward(self, x,y, mask = None):
        batch_size, L, d_model = x.size()
        kv = self.kv_layer(x) #1024
        q = self.qlayer(y) #512
        kv = kv.reshape(batch_size, L, self.num_heads, 2*self.head_dim)
        q = q.reshape(batch_size, L, self.num_heads, self.head_dim)
        kv = kv.permute(0,2,1,3)
        q = q.permute(0,2,1,3)
        k, v = kv.chunk(2, dim=-1)
        out, attention = self.scaled_dot_product_attention(q,k,v,mask)
        out = out.permute(0,2,1,3)
        out = out.reshape(batch_size, L, self.d_model)
        out = self.linear(out)
        # print("MultiHeadCrossAttention")
        return out
    
class MultiHeadAttention(nn.Module):
    def __init__(self, d_model, num_heads):
        super().__init__()
        self.d_model = d_model
        self.num_heads = num_heads
        self.head_dim = d_model // num_heads
        self.qkv_layer = nn.Linear(d_model, 3*d_model)
        self.softmax = nn.Softmax(dim = -1)
        self.linear = nn.Linear(d_model, d_model)

    def scaled_dot_product_attention(self,q,k,v,mask=None):
        d_k = q.shape[-1]
        k_t = k.transpose(-2,-1)
        scaled = torch.matmul(q, k_t) / math.sqrt(d_k)
        if mask is not None:
            scaled = (scaled.permute(1,0,2,3) + mask).permute(1,0,2,3)
        attention = self.softmax(scaled)
        out = torch.matmul(attention, v)
        return out, attention
    
    def forward(self, x, mask = None):
        batch_size, L, d_model = x.size()
        qkv = self.qkv_layer(x) #1536
        qkv = qkv.reshape(batch_size, L, self.num_heads, 3*self.head_dim)
        qkv = qkv.permute(0,2,1,3)
        q, k, v = qkv.chunk(3, dim=-1)
        out, attention = self.scaled_dot_product_attention(q,k,v,mask)
        out = out.permute(0,2,1,3)
        out = out.reshape(batch_size, L, self.d_model)
        out = self.linear(out)
        # print("MultiHeadAttention")
        return out

class PositionWiseFeedForward(nn.Module):
    def __init__(self, d_model, hidden,drop_prob):
        super(PositionWiseFeedForward, self).__init__()
        self.layer1 = nn.Linear(d_model, hidden)
        self.layer2 = nn.Linear(hidden, d_model)
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(p = drop_prob)
    
    def forward(self, x):
        x = self.layer1(x)
        x = self.relu(x)
        x = self.dropout(x)
        x = self.layer2(x)
        # print("PositionWiseFeedForward")
        return x


class LayerNormalisation(nn.Module):
    def __init__(self,parameters_shape,eps=1e-5):
        super().__init__()
        self.parameters_shape = parameters_shape
        self.eps = eps
        self.gamma = nn.Parameter(torch.ones(parameters_shape))
        self.beta = nn.Parameter(torch.zeros(parameters_shape))

    def forward(self, inputs):
        dims = [-(i+1) for i in range(len(self.parameters_shape))]
        mean = inputs.mean(dim=dims, keepdim=True)
        var = ((inputs - mean)**2).mean(dim=dims, keepdim=True)
        std = (var + self.eps).sqrt()
        y = (inputs - mean)/std
        out = self.gamma * y + self.beta
        # print("LayerNormalisation")
        return out

class DecoderLayer(nn.Module):
    def __init__(self, d_model,ffn_hidden, num_heads,  drop_prob):
        super(DecoderLayer,self).__init__()
        self.attention = MultiHeadAttention(d_model, num_heads)
        self.cross_attention = MultiHeadCrossAttention(d_model, num_heads)
        self.norm1 = LayerNormalisation(parameters_shape=[d_model])
        self.dropout1 = nn.Dropout(p = drop_prob)
        self.ffn = PositionWiseFeedForward(d_model, ffn_hidden, drop_prob)
        self.norm2 = LayerNormalisation([d_model])
        self.dropout2 = nn.Dropout(p=drop_prob)
        self.norm3 = LayerNormalisation([d_model])
        self.dropout3 = nn.Dropout(p=drop_prob)

    def forward(self,x,y,self_attention_mask, cross_attention_mask):
        residual_y = y
        y = self.attention(y, self_attention_mask)
        y = self.dropout1(y)
        y = self.norm1(y + residual_y)
        residual_y = y
        y = self.cross_attention(x,y, cross_attention_mask)
        y = self.dropout2(y)
        y = self.norm2(y + residual_y)
        residual_y = y
        y = self.ffn(y)
        y = self.dropout3(y)
        y = self.norm3(y + residual_y)
        return y

class SequentialDecoder(nn.Sequential):
    # def forward(self, *inputs):
    def forward(self, x,y, self_mask, cross_mask):
        # x,y,self_mask, cross_mask = inputs
        for module in self._modules.values():
            y = module(x,y,self_mask, cross_mask)
        return y

class Decoder(nn.Module):
    def __init__(self, d_model, ffn_hidden, num_heads, drop_prob, num_layers, max_sq_len, language_to_index, START, END, PADDING):
        super().__init__()
        self.sentence_embedding = SentenceEmbedding(max_sq_len, d_model, language_to_index, START, END, PADDING)
        self.layers = SequentialDecoder(*[DecoderLayer(d_model, ffn_hidden, num_heads, drop_prob) for _ in range(num_layers)])

    def forward(self,x,y,self_attention_mask, cross_attention_mask, START, END):
        y = self.sentence_embedding(y,START, END)
        # try:
        #     y = self.layers(x,y,self_attention_mask, cross_attention_mask)
        # except:
        #     print("Error is in decoder")
        y = self.layers(x,y,self_attention_mask, cross_attention_mask)
        return y

class EncoderLayer(nn.Module):
    def __init__(self, d_model, ffn_hidden, num_heads, drop_prob):
        super(EncoderLayer, self).__init__()
        self.attention = MultiHeadAttention(d_model, num_heads)
        self.norm1 = LayerNormalisation(parameters_shape=[d_model])
        self.dropout1 = nn.Dropout(p = drop_prob)
        self.ffn = PositionWiseFeedForward(d_model, ffn_hidden, drop_prob)
        self.norm2 = LayerNormalisation([d_model])
        self.dropout2 = nn.Dropout(p=drop_prob)
    
    def forward(self,x, self_attention_mask):
        residual_x =x
        x = self.attention(x, mask= self_attention_mask)
        x = self.dropout1(x)
        x = self.norm1(x+residual_x)
        residual_x = x
        x = self.ffn(x)
        x = self.dropout2(x)
        x = self.norm2(x+residual_x)
        return x

class SequentialEncoder(nn.Sequential):
    # def forward(self, *inputs):
    def forward(self, x, self_mask):
        for module in self._modules.values():
            x = module(x, self_mask)
        return x

class Encoder(nn.Module):
    def __init__(self, d_model, ffn_hidden, num_heads, drop_prob, num_layers, max_sq_len, language_to_index, START, END, PADDING):
        super().__init__()
        self.sentence_embedding = SentenceEmbedding(max_sq_len, d_model, language_to_index, START, END, PADDING)
        self.layers = SequentialEncoder(*[EncoderLayer(d_model, ffn_hidden, num_heads, drop_prob) for _ in range(num_layers)])
        
    def forward(self,x, self_attention_mask, START, END):
        x = self.sentence_embedding(x, START, END)
        # try:
        #     x = self.layers(x, self_attention_mask)
        # except:
        #     print("Error is in encoder")
        # return x
        x = self.layers(x, self_attention_mask)
        return x

class  Transformer(nn.Module):
    def __init__(self, d_model, ffn_hidden, num_heads, drop_prob, num_layers, max_sq_len, hindi_vocab_size, english_to_index, hindi_to_index, START, END, PADDING):
        super().__init__()
        self.d_model = d_model
        self.ffn_hidden = ffn_hidden
        self.num_heads = num_heads
        self.drop_prob = drop_prob
        self.num_layers = num_layers
        self.encoder = Encoder(d_model, ffn_hidden, num_heads, drop_prob, num_layers, max_sq_len, english_to_index, START, END, PADDING)
        self.decoder = Decoder(d_model, ffn_hidden, num_heads, drop_prob, num_layers, max_sq_len, hindi_to_index, START, END, PADDING)
        self.linear = nn.Linear(d_model, hindi_vocab_size)
    
    def forward(self, x, y,encoder_self_attention_mask = None, decoder_self_attention_mask = None, decoder_cross_attention_mask = None, enc_start = False, dec_start= False, dec_end = False, enc_end = False):
        x = self.encoder(x, encoder_self_attention_mask, enc_start, enc_end)
        out = self.decoder(x, y , decoder_self_attention_mask, decoder_cross_attention_mask, dec_start, dec_end)
        out = self.linear(out)
        return out
 

In [2]:
START = '<START>'
PADDING = '<PADDING>'
END = '<END>'

characters_hindi = [START,'अ', 'आ', 'इ', 'ई', 'उ', 'ऊ', 'ऋ', 'ए', 'ऐ', 'ओ', 'औ', 
                    'क', 'ख', 'ग', 'घ', 'ङ', 'च', 'छ', 'ज', 'झ', 'ञ', 'ट', 
                    'ठ', 'ड', 'ढ', 'ण', 'त', 'थ', 'द', 'ध', 'न', 'प', 'फ', 
                    'ब', 'भ', 'म', 'य', 'र', 'ल', 'व', 'श', 'ष', 'स', 'ह','ॉ',
                    'ा', 'ि', 'ी', 'ु', 'ू', 'ृ', 'े', 'ै', 'ो', 'ौ', 'ं', 
                    'ः', '्', 'ॐ', '।', '॥', 'ँ', '़', 'ऽ', '०', '१', '२', 
                    '३', '४', '५', '६', '७', '८', '९', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ',', '.', ':', '!',
                    '?', '(', ')', '‘', '’', '“', '”', '-', '_',' ', PADDING, END]


characters_english = [START,'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 
              'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z',
              'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 
              'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z',
              '.', ',', ':', ';', '!', '?', "'", '"', '(', ')', '[', ']', '{', '}', 
              '-', '_', '/', '\\', '@', '#', '$', '%', '^', '&','|', '~', '`', 
              '0','1', '2', '3', '4', '5', '6', '7', '8', '9',' ', PADDING, END]


In [3]:
hindi_vocab_size = len(characters_hindi)
enlish_vocab_size = len(characters_english)

In [4]:
with open('D:\\ML\\transformer_scratch\\en-hi\\train.en') as enfile:
    english = [next(enfile) for _ in range(10000)]
with open('D:\\ML\\transformer_scratch\\en-hi\\train.hi', encoding = 'utf-8') as hifile:
    hindi =  [next(hifile) for _ in range(10000)]

In [5]:
index_to_hindi = {k:v for k,v in enumerate(characters_hindi)}
hindi_to_index = {v:k for k,v in enumerate(characters_hindi)}
index_to_english = {k:v for k,v in enumerate(characters_english)}
english_to_index = {v:k for k,v in enumerate(characters_english)}

In [6]:
max_sq_len = 200

In [7]:
def is_valid_token(sentence, vocab):
    for token in list(set(sentence)):
        if token not in vocab:
            return False
    return True

def isvalidlength(sentence, max_sq_len):
    return len(list(sentence))< (max_sq_len-1)

valid_index = []
for i in range(10000):
    if hindi[i][-1]=='\n':
            hindi[i] = hindi[i][:-1]
    if english[i][-1]=='\n':
        english[i] = english[i][:-1]
    hsen, esen = hindi[i], english[i]
    if(is_valid_token(hsen,characters_hindi) and is_valid_token(esen, characters_english) and isvalidlength(esen,max_sq_len) and isvalidlength(hsen,max_sq_len)):
        valid_index.append(i)
print(len(valid_index))

7310


In [8]:
import torch
d_model = 512
batch_size = 30
ffn_hidden = 2048
num_heads = 8
drop_prob =0.1
num_layers = 1
max_sq_len = 200
hindi_vocab_size = len(characters_hindi)
transformer = Transformer(d_model, ffn_hidden, num_heads, drop_prob, num_layers, max_sq_len, hindi_vocab_size, english_to_index, hindi_to_index, START, END, PADDING)

In [9]:
transformer

Transformer(
  (encoder): Encoder(
    (sentence_embedding): SentenceEmbedding(
      (embedding): Embedding(93, 512)
      (position_encoder): PositionalEncoding()
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (layers): SequentialEncoder(
      (0): EncoderLayer(
        (attention): MultiHeadAttention(
          (qkv_layer): Linear(in_features=512, out_features=1536, bias=True)
          (softmax): Softmax(dim=-1)
          (linear): Linear(in_features=512, out_features=512, bias=True)
        )
        (norm1): LayerNormalisation()
        (dropout1): Dropout(p=0.1, inplace=False)
        (ffn): PositionWiseFeedForward(
          (layer1): Linear(in_features=512, out_features=2048, bias=True)
          (layer2): Linear(in_features=2048, out_features=512, bias=True)
          (relu): ReLU()
          (dropout): Dropout(p=0.1, inplace=False)
        )
        (norm2): LayerNormalisation()
        (dropout2): Dropout(p=0.1, inplace=False)
      )
    )
  )
  (decoder): Decod

In [10]:
from torch.utils.data import Dataset, DataLoader

class TextDataset(Dataset):

    def __init__(self, english_sentences, hindi_sentences):
        self.english_sentences = english_sentences
        self.hindi_sentences = hindi_sentences

    def __len__(self):
        return len(self.english_sentences)

    def __getitem__(self, idx):
        return self.english_sentences[idx], self.hindi_sentences[idx]

In [11]:
english_sentences = [english[i] for i in valid_index]
hindi_sentences = [hindi[i] for i in valid_index]

In [12]:
dataset = TextDataset(english_sentences, hindi_sentences)

In [13]:
dataset[1]

('The Congress leader represents Sivaganga Lok Sabha segment from Tamil Nadu.',
 'कांग्रेस नेता तमिलनाडु से शिवगंगा लोकसभा क्षेत्र का प्रतिनिधित्व करते हैं.')

In [14]:
train_loader = DataLoader(dataset, batch_size)
iterator = iter(train_loader)

In [15]:
for batch_num, batch in enumerate(iterator):
    print(batch)
    if batch_num > 3:
        break

[('In reply, Pakistan got off to a solid start.', 'The Congress leader represents Sivaganga Lok Sabha segment from Tamil Nadu.', 'Prompt the user about connection attempts', 'Further, the Minister announced that Deposit Insurance and Credit Guarantee Corporation (DICGC) has been permitted to increase Deposit Insurance coverage to Rs.', 'Therefore, brothers, be more diligent to make your calling and election sure. For if you do these things, you will never stumble.', 'The review committee meeting chaired by the District Judges will be attended by Collectors, SPs, Superintendents of jails and Secretary of District Legal Services Authority.', 'Police is present on the spot.', 'Prime Minister Narendra Modi is slated to campaign for BJP in second phase of Assembly elections in Udhampur and Poonch district of Jammu and Kashmir tomorrow', 'It has a battery backup of 3050mAh.', 'All 176 passengers died in the incident.', 'Not invited', 'Sunny Leone plays a paranormal expert in the series.', 'T

In [16]:
from torch import nn

criterian = nn.CrossEntropyLoss(ignore_index=hindi_to_index[PADDING],
                                reduction='none')

# When computing the loss, we are ignoring cases when the label is the padding token
for params in transformer.parameters():
    if params.dim() > 1:
        nn.init.xavier_uniform_(params)

optim = torch.optim.Adam(transformer.parameters(), lr=1e-4)
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

c:\Python312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [17]:
NEG_INFTY = -1e9

def create_masks(eng_batch, hin_batch):
    num_sen = len(eng_batch)
    look_ahead_mask = torch.full([max_sq_len, max_sq_len],True)
    look_ahead_mask = torch.triu(look_ahead_mask, diagonal=1)
    encoder_padding_mask = torch.full([num_sen, max_sq_len, max_sq_len], False)
    decoder_padding_mask_self_attention = torch.full([num_sen, max_sq_len, max_sq_len], False)
    decoder_padding_mask_cross_attention = torch.full([num_sen, max_sq_len, max_sq_len], False)

    for i in range(num_sen):
        eng_sen_len, hin_sen_len = len(eng_batch[i]), len(hin_batch[i])
        eng_padding_char = np.arange(eng_sen_len, max_sq_len)
        hin_padding_char = np.arange(hin_sen_len, max_sq_len)
        encoder_padding_mask[i,:,eng_padding_char] = True
        encoder_padding_mask[i,eng_padding_char, :] = True
        decoder_padding_mask_self_attention[i, :, hin_padding_char] = True
        decoder_padding_mask_self_attention[i, hin_padding_char, :] = True
        decoder_padding_mask_cross_attention[i, :, eng_padding_char] = True
        decoder_padding_mask_cross_attention[i, hin_padding_char, :] = True

    
    encoder_self_attention_mask = torch.where(encoder_padding_mask, NEG_INFTY,0)
    decoder_self_attention_mask = torch.where(look_ahead_mask + decoder_padding_mask_self_attention, NEG_INFTY, 0)
    decoder_cross_attention_mask = torch.where(decoder_padding_mask_cross_attention, NEG_INFTY, 0)
    return encoder_self_attention_mask, decoder_self_attention_mask, decoder_cross_attention_mask

In [18]:
import numpy as np
from tqdm import tqdm
transformer.train()
transformer.to(device)
total_loss = 0
num_epochs = 10

for epoch in range(num_epochs):
    print(f"Epoch {epoch}")
    iterator = iter(train_loader)
    for batch_num, batch in tqdm(enumerate(iterator)):
        transformer.train()
        eng_batch, hn_batch = batch
        encoder_self_attention_mask, decoder_self_attention_mask, decoder_cross_attention_mask = create_masks(eng_batch, hn_batch)
        optim.zero_grad()
        hn_predictions = transformer(eng_batch, hn_batch, encoder_self_attention_mask.to(device), decoder_self_attention_mask.to(device), decoder_cross_attention_mask.to(device),False,False,True,True)
        labels = transformer.decoder.sentence_embedding.batch_tokenisation(hn_batch, start_token=False, end_token=True)
        loss = criterian(hn_predictions.view(-1, hindi_vocab_size).to(device),
            labels.view(-1).to(device)
        ).to(device)
        valid_indicies = torch.where(labels.view(-1) == hindi_to_index[PADDING], False, True)
        loss = loss.sum() / valid_indicies.sum()
        loss.backward()
        optim.step()
        #train_losses.append(loss.item())
        if batch_num % 100 == 0:
            print(f"Iteration {batch_num} : {loss.item()}")
            print(f"English: {eng_batch[0]}")
            print(f"Hindi Translation: {hn_batch[0]}")
            hn_sentence_predicted = torch.argmax(hn_predictions[0], axis=1)
            predicted_sentence = ""
            for idx in hn_sentence_predicted:
              if idx == hindi_to_index[END]:
                break
              predicted_sentence += index_to_hindi[idx.item()]
            print(f"Hindi Prediction: {predicted_sentence}")


            transformer.eval()
            hn_sentence = ("",)
            eng_sentence = ("What is your name?",)
            for word_counter in range(max_sq_len):
                encoder_self_attention_mask, decoder_self_attention_mask, decoder_cross_attention_mask= create_masks(eng_sentence, hn_sentence)
                predictions = transformer(eng_sentence,
                                          hn_sentence,
                                          encoder_self_attention_mask.to(device), 
                                          decoder_self_attention_mask.to(device), 
                                          decoder_cross_attention_mask.to(device),False,False,True,False)
                next_token_prob_distribution = predictions[0][word_counter] # not actual probs
                next_token_index = torch.argmax(next_token_prob_distribution).item()
                next_token = index_to_hindi[next_token_index]
                hn_sentence = (hn_sentence[0] + next_token, )
                if next_token == END:
                  break
            
            print(f"Evaluation translation (What is your name?) : {hn_sentence}")
            print("-------------------------------------------")

Epoch 0


0it [00:00, ?it/s]

Iteration 0 : 5.7827839851379395
English: In reply, Pakistan got off to a solid start.
Hindi Translation: जिसके जवाब में पाक ने अच्छी शुरुआत की थी.
Hindi Prediction: छृ?उउउउँरऽ६े६6ररउँर”ँचछँ-६-६


1it [00:18, 18.02s/it]

Evaluation translation (What is your name?) : ('ररो?  ररररररररररररररचच         कक   र--------ण     गणणण  न   णणणणणणणणणण                          णणणण                                                           सनन                      रररररर         ',)
-------------------------------------------


...

101it [06:01,  3.51s/it]

Iteration 100 : 0.0006300988607108593
English: People worship Goddess Lakshmi on this day.
Hindi Translation: इस दिन लक्ष्मी का पूजन किया जाता है।
Hindi Prediction: इस दिन लक्ष्मी का पूजन किया जाता है।
Evaluation translation (What is your name?) : ('<END>',)
-------------------------------------------


201it [11:53,  3.62s/it]

Iteration 200 : 0.0008362542721442878
English: The quicker this is done the better for the country.
Hindi Translation: वैसे जितनी जल्दी कार्रवाई होगी, उतना ही देश के लिए बेहतर होगा.
Hindi Prediction: वैसे जितनी जल्दी कार्रवाई होगी, उतना ही देश के लिए बेहतर होगा.
Evaluation translation (What is your name?) : ('<END>',)
-------------------------------------------


244it [14:21,  3.53s/it]


In [22]:
eng_sentence = ("good",)
for word_counter in range(max_sq_len):
    encoder_self_attention_mask, decoder_self_attention_mask, decoder_cross_attention_mask= create_masks(eng_sentence, hn_sentence)
    predictions = transformer(eng_sentence,
                                hn_sentence,
                                encoder_self_attention_mask.to(device), 
                                decoder_self_attention_mask.to(device), 
                                decoder_cross_attention_mask.to(device),False,False,True,False)
    next_token_prob_distribution = predictions[0][word_counter] # not actual probs
    next_token_index = torch.argmax(next_token_prob_distribution).item()
    next_token = index_to_hindi[next_token_index]
    hn_sentence = (hn_sentence[0] + next_token, )
    if next_token == END:
        break

KeyError: '<'